In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
import time

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [2]:
def obtener_rmse(col_true, col_pred):
    return mean_squared_error(col_true, col_pred)**0.5

In [2]:
#propiedades = pd.read_csv('set_datos_propiedades.csv')
propiedades = pd.read_csv('/home/agustin/Escritorio/escritorio/fiuba/Organizacion de datos/datos para el tp2/set_datos_propiedades.csv')

In [4]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull()),\
                             ['place_name_encoded', 'property_type_encoded','price_aprox_usd','superficie',\
                             'Year','Month','seguridad','aire','gimnasio','cochera','pileta']]

In [5]:
propiedades.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1132495 entries, 0 to 1413024
Data columns (total 11 columns):
place_name_encoded       1132495 non-null int64
property_type_encoded    1132495 non-null int64
price_aprox_usd          1132495 non-null float64
superficie               1132495 non-null float64
Year                     1132495 non-null int64
Month                    1132495 non-null int64
seguridad                1132495 non-null bool
aire                     1132495 non-null bool
gimnasio                 1132495 non-null bool
cochera                  1132495 non-null bool
pileta                   1132495 non-null bool
dtypes: bool(5), float64(2), int64(4)
memory usage: 65.9 MB


# Gradient boosting regressor

In [6]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [7]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [8]:
gradient = GradientBoostingRegressor()
gradient.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: gradient.predict([x])[0],axis = 1)
precision = gradient.score(set_pruebas.loc[:,columnas], set_pruebas.price_aprox_usd) * 100
error = obtener_rmse(set_pruebas.price_aprox_usd, set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 49.59 % , error = 263717.911261235


#### Al correrlo varias veces con distintos valores, fue mejorando la precision. Despues agrego un for con varios valores para ver como mejora

## Ahora que tenemos una intuicion, probamos cambiando los parametros

In [9]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [10]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']


In [15]:
res = []
loss = ['ls', 'lad', 'huber', 'quantile']
n_estimators = [300, 500, 1000, 2000]

for l in loss:
    for n in n_estimators:
        gradient = GradientBoostingRegressor(loss = l, n_estimators = n)
        gradient.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
        set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: gradient.predict([x])[0],axis = 1)
        precision = gradient.score(set_pruebas.loc[:,columnas],set_pruebas.loc[:,'price_aprox_usd']) * 100
        error = obtener_rmse(set_pruebas.price_aprox_usd, set_pruebas.resultado)
        res.append((l, n, precision, error))
        print(l,' - ', n)

ls  -  300
ls  -  500
ls  -  1000
ls  -  2000
lad  -  300
lad  -  500
lad  -  1000
lad  -  2000
huber  -  300
huber  -  500
huber  -  1000
huber  -  2000
quantile  -  300
quantile  -  500
quantile  -  1000
quantile  -  2000


In [18]:
for r in res

    print("Loss = {}, n_estimators = {}, precision = {:.2f} % , error = {}".format(r[0],r[1],r[2],r[3]))

Loss = ls, n_estimators = 300, precision = 55.15 % , error = 248737.31486991755
Loss = ls, n_estimators = 500, precision = 57.50 % , error = 242135.99581654233
Loss = ls, n_estimators = 1000, precision = 60.48 % , error = 233486.84277362804
Loss = ls, n_estimators = 2000, precision = 62.91 % , error = 226215.10776356
Loss = lad, n_estimators = 300, precision = 47.35 % , error = 269518.0659149018
Loss = lad, n_estimators = 500, precision = 49.66 % , error = 263524.37826822215
Loss = lad, n_estimators = 1000, precision = 50.67 % , error = 260861.96525934708
Loss = lad, n_estimators = 2000, precision = 51.67 % , error = 258216.88529997237
Loss = huber, n_estimators = 300, precision = 51.41 % , error = 258917.110961643
Loss = huber, n_estimators = 500, precision = 46.40 % , error = 271928.52928042517
Loss = huber, n_estimators = 1000, precision = 47.81 % , error = 268320.914070446
Loss = huber, n_estimators = 2000, precision = 48.50 % , error = 266545.7580620435
Loss = quantile, n_estimato

In [32]:
min_error = float('inf')
max_precision = 0
tupla_min_error = ()
tupla_max_precision = ()
for r in res:
    if r[3] < min_error:
        min_error = r[3]
        tupla_min_error = r
    if abs(r[2]) > max_precision:
        max_precision = r[2]
        tupla_max_precision = r
        
print("Mayor precision = Loss = {}, n_estimators = {}, precision = {:.2f} % , error = {}".\
              format(tupla_max_precision[0],tupla_max_precision[1],tupla_max_precision[2],tupla_max_precision[3]))
print("Menor error = Loss = {}, n_estimators = {}, precision = {:.2f} % , error = {}".\
              format(tupla_min_error[0],tupla_min_error[1],tupla_min_error[2],tupla_min_error[3]))

Mayor precision = Loss = ls, n_estimators = 2000, precision = 62.91 % , error = 226215.10776356
Menor error = Loss = ls, n_estimators = 2000, precision = 62.91 % , error = 226215.10776356


## Usamos ls como loss

In [33]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [34]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [36]:
loss = 'ls'
n_est = 2000

In [37]:
gradient = GradientBoostingRegressor(loss = loss, n_estimators = n_est, warm_start = True)
for i in range(10):
    gradient.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: gradient.predict([x])[0],axis = 1)
    precision = gradient.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
    error = obtener_rmse(set_pruebas.price_aprox_usd,set_pruebas.resultado)
    print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 62.91 % , error = 226215.10776355996
Precision = 62.91 % , error = 226215.10776355996
Precision = 62.91 % , error = 226215.10776355996
Precision = 62.91 % , error = 226215.10776355996
Precision = 62.91 % , error = 226215.10776355996
Precision = 62.91 % , error = 226215.10776355996
Precision = 62.91 % , error = 226215.10776355996
Precision = 62.91 % , error = 226215.10776355996
Precision = 62.91 % , error = 226215.10776355996
Precision = 62.91 % , error = 226215.10776355996


#### No se obtuvo mejora utilizando warm_start = True, por lo que realizar varias iteraciones con los mismos datos no mejorará la precisión

In [38]:
gradient = GradientBoostingRegressor(loss = loss, n_estimators = 10000, warm_start = True)
gradient.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: gradient.predict([x])[0],axis = 1)
precision = gradient.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = obtener_rmse(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 68.10 % , error = 209782.67862516703


In [66]:
set_pruebas.resultado.describe()

count    2.000000e+04
mean     2.645389e+05
std      2.998174e+05
min      5.833006e+02
25%      1.081312e+05
50%      1.716801e+05
75%      2.996141e+05
max      6.980550e+06
Name: resultado, dtype: float64

In [48]:
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,"resultado"].apply(lambda x: abs(x))

In [51]:
obtener_rmse(set_pruebas.price_aprox_usd,set_pruebas.resultado)

209673.22218335391

## Vario la profundidad 

In [77]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [78]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [79]:
depth = [1, 3, 5, 10, 20]
learning = [0.1, 0.3, 0.5, 0.7]

In [84]:
res = []
for d in depth:
    for l in learning:
        gradient = GradientBoostingRegressor(loss = "ls", max_depth = d, learning_rate = l)
        gradient.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
        set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: gradient.predict([x])[0],axis = 1)
        precision = gradient.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
        error = obtener_rmse(set_pruebas.price_aprox_usd,set_pruebas.resultado)
        res.append((d, l, precision, error))
        print(d, " - ", l)

1  -  0.1
1  -  0.3
1  -  0.5
1  -  0.7
3  -  0.1
3  -  0.3
3  -  0.5
3  -  0.7
5  -  0.1
5  -  0.3
5  -  0.5
5  -  0.7
10  -  0.1
10  -  0.3
10  -  0.5
10  -  0.7
20  -  0.1


Los casos faltantes de profundidad 20 se descartaron debido al elevado tiempo de aprendizaje

In [82]:
for r in res:
    print("max_depth = {}, learning_rate = {}, precision = {:.2f} % , error = {}".format(r[0],r[1],r[2],r[3]))

max_depth = 1, learning_rate = 0.1, precision = 36.24 % , error = 296585.3952956776
max_depth = 1, learning_rate = 0.3, precision = 38.77 % , error = 290645.3508711553
max_depth = 1, learning_rate = 0.5, precision = 39.96 % , error = 287808.51835432526
max_depth = 1, learning_rate = 0.7, precision = 40.81 % , error = 285757.0809324506
max_depth = 3, learning_rate = 0.1, precision = 49.59 % , error = 263717.91126123496
max_depth = 3, learning_rate = 0.3, precision = 55.44 % , error = 247942.04557652943
max_depth = 3, learning_rate = 0.5, precision = 57.41 % , error = 242387.34201665278
max_depth = 3, learning_rate = 0.7, precision = 58.25 % , error = 239982.7016595514
max_depth = 5, learning_rate = 0.1, precision = 58.86 % , error = 238243.53268510482
max_depth = 5, learning_rate = 0.3, precision = 65.91 % , error = 216863.74408280538
max_depth = 5, learning_rate = 0.5, precision = 68.59 % , error = 208176.2512388309
max_depth = 5, learning_rate = 0.7, precision = 69.94 % , error = 2036

## Tomo un millon de estimadores para ver cuanto mejora la precision

In [86]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [87]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [88]:
gradient = GradientBoostingRegressor(loss = loss, n_estimators = 2000, learning_rate= 0.1, max_depth = 10)
gradient.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: gradient.predict([x])[0],axis = 1)
precision = gradient.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = obtener_rmse(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 90.58 % , error = 114027.97059902015


In [89]:
set_pruebas.resultado.describe()

count    2.000000e+04
mean     2.657068e+05
std      3.517090e+05
min     -1.695101e+04
25%      9.974896e+04
50%      1.583140e+05
75%      2.900442e+05
max      1.109959e+07
Name: resultado, dtype: float64

In [90]:
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,"resultado"].apply(lambda x: abs(x))

In [92]:
set_pruebas.resultado.describe()

count    2.000000e+04
mean     2.657085e+05
std      3.517077e+05
min      9.234466e+03
25%      9.974896e+04
50%      1.583140e+05
75%      2.900442e+05
max      1.109959e+07
Name: resultado, dtype: float64

In [93]:
obtener_rmse(set_pruebas.price_aprox_usd,set_pruebas.resultado)

114027.37597125868

# calculo con los verdaderos datos a analizar

In [94]:
analizar = pd.read_csv("properati_dataset_modificado.csv")

In [95]:
analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: gradient.predict([x])[0],axis = 1)

In [96]:
analizar.price_usd.describe()

count    1.416600e+04
mean     2.373848e+05
std      3.434490e+05
min     -5.127444e+05
25%      1.054205e+05
50%      1.572446e+05
75%      2.630949e+05
max      2.771613e+07
Name: price_usd, dtype: float64

In [97]:
# Las predicciones con precio negativo las convierto en valores positivos
analizar.loc[:,'price_usd'] = analizar.loc[:,"price_usd"].apply(lambda x: abs(x))

In [98]:
analizar.price_usd.describe()

count    1.416600e+04
mean     2.377970e+05
std      3.431637e+05
min      1.270784e+02
25%      1.055294e+05
50%      1.573691e+05
75%      2.632307e+05
max      2.771613e+07
Name: price_usd, dtype: float64

In [99]:
resultado = analizar.loc[:,['id','price_usd']]

In [100]:
resultado.to_csv('Gradient_boosting_2.csv', index = False)

# Ahora uso lat y lon en lugar de place name

In [3]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()),\
                             ['lat', 'lon', 'property_type_encoded','price_aprox_usd','superficie',\
                             'Year','Month','seguridad','aire','gimnasio','cochera','pileta']]

In [4]:
columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [5]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [6]:
gradient = GradientBoostingRegressor()
gradient.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: gradient.predict([x])[0],axis = 1)
precision = gradient.score(set_pruebas.loc[:,columnas], set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 55.02 % , error = 52075495716.7


In [15]:
res = []

In [16]:
loss = ['ls', 'lad', 'huber', 'quantile']
n_estimators = [300, 500, 1000, 2000, 5000]
d = 3
lr = 0.1

for l in loss:
    for n in n_estimators: 
        gradient = GradientBoostingRegressor(loss = l, n_estimators = n, max_depth = d, learning_rate = lr)
        gradient.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
        set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: gradient.predict([x])[0],axis = 1)
        precision = gradient.score(set_pruebas.loc[:,columnas],set_pruebas.loc[:,'price_aprox_usd']) * 100
        error = mean_squared_error(set_pruebas.price_aprox_usd, set_pruebas.resultado)
        res.append((l, n, d, lr, precision, error))
        print(l,' - ', n, '-', d, '-', lr)

('ls', ' - ', 300, '-', 3, '-', 0.1)
('ls', ' - ', 500, '-', 3, '-', 0.1)
('ls', ' - ', 1000, '-', 3, '-', 0.1)
('ls', ' - ', 2000, '-', 3, '-', 0.1)
('ls', ' - ', 5000, '-', 3, '-', 0.1)
('lad', ' - ', 300, '-', 3, '-', 0.1)
('lad', ' - ', 500, '-', 3, '-', 0.1)
('lad', ' - ', 1000, '-', 3, '-', 0.1)
('lad', ' - ', 2000, '-', 3, '-', 0.1)
('lad', ' - ', 5000, '-', 3, '-', 0.1)
('huber', ' - ', 300, '-', 3, '-', 0.1)
('huber', ' - ', 500, '-', 3, '-', 0.1)
('huber', ' - ', 1000, '-', 3, '-', 0.1)
('huber', ' - ', 2000, '-', 3, '-', 0.1)
('huber', ' - ', 5000, '-', 3, '-', 0.1)
('quantile', ' - ', 300, '-', 3, '-', 0.1)
('quantile', ' - ', 500, '-', 3, '-', 0.1)
('quantile', ' - ', 1000, '-', 3, '-', 0.1)
('quantile', ' - ', 2000, '-', 3, '-', 0.1)
('quantile', ' - ', 5000, '-', 3, '-', 0.1)


In [17]:
min_error = float('inf')
max_precision = 0
tupla_min_error = ()
tupla_max_precision = ()
for r in res:
    if r[5] < min_error:
        min_error = r[5]
        tupla_min_error = r
    if abs(r[4]) > max_precision:
        max_precision = r[4]
        tupla_max_precision = r
        
print("Mayor precision = Loss = {}, n_estimators = {}, max_depth = {}, learning_rate = {} precision = {:.2f} % , error = {}".\
              format(tupla_max_precision[0],tupla_max_precision[1],tupla_max_precision[2],tupla_max_precision[3], tupla_max_precision[4], tupla_max_precision[5]))
print("Menor error = Loss = {}, n_estimators = {}, max_depth = {}, learning_rate = {} precision = {:.2f} % , error = {}".\
              format(tupla_min_error[0],tupla_min_error[1],tupla_min_error[2],tupla_min_error[3], tupla_min_error[4], tupla_min_error[5]))

Mayor precision = Loss = ls, n_estimators = 5000, max_depth = 3, learning_rate = 0.1 precision = 78.73 % , error = 24626934120.9
Menor error = Loss = ls, n_estimators = 5000, max_depth = 3, learning_rate = 0.1 precision = 78.73 % , error = 24626934120.9


In [19]:
l = 'ls'
n = 100 #Utilizo 100 para que sea mas rapido
depth = [3, 5, 10, 20]
learning = [0.3, 0.5, 0.7]
for d in depth:
    for lr in learning:
        gradient = GradientBoostingRegressor(loss = l, n_estimators = n, max_depth = d, learning_rate = lr)
        gradient.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
        set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: gradient.predict([x])[0],axis = 1)
        precision = gradient.score(set_pruebas.loc[:,columnas],set_pruebas.loc[:,'price_aprox_usd']) * 100
        error = mean_squared_error(set_pruebas.price_aprox_usd, set_pruebas.resultado)
        res.append((l, n, d, lr, precision, error))
        print(l,' - ', n, '-', d, '-', lr)

('ls', ' - ', 100, '-', 3, '-', 0.3)
('ls', ' - ', 100, '-', 3, '-', 0.5)
('ls', ' - ', 100, '-', 3, '-', 0.7)
('ls', ' - ', 100, '-', 5, '-', 0.3)
('ls', ' - ', 100, '-', 5, '-', 0.5)
('ls', ' - ', 100, '-', 5, '-', 0.7)
('ls', ' - ', 100, '-', 10, '-', 0.3)
('ls', ' - ', 100, '-', 10, '-', 0.5)
('ls', ' - ', 100, '-', 10, '-', 0.7)
('ls', ' - ', 100, '-', 20, '-', 0.3)
('ls', ' - ', 100, '-', 20, '-', 0.5)
('ls', ' - ', 100, '-', 20, '-', 0.7)


In [21]:
for r in res:
    print("Loss = {}, n_estimators = {}, max_depth = {}, learning_rate = {} precision = {:.2f} % , error = {}"\
          .format(r[0],r[1],r[2],r[3], r[4], r[5]))

Loss = ls, n_estimators = 300, max_depth = 3, learning_rate = 0.1 precision = 60.32 % , error = 45940935462.6
Loss = ls, n_estimators = 500, max_depth = 3, learning_rate = 0.1 precision = 62.87 % , error = 42994100031.3
Loss = ls, n_estimators = 1000, max_depth = 3, learning_rate = 0.1 precision = 69.18 % , error = 35688017721.5
Loss = ls, n_estimators = 2000, max_depth = 3, learning_rate = 0.1 precision = 73.62 % , error = 30540892887.9
Loss = ls, n_estimators = 5000, max_depth = 3, learning_rate = 0.1 precision = 78.73 % , error = 24626934120.9
Loss = lad, n_estimators = 300, max_depth = 3, learning_rate = 0.1 precision = 51.36 % , error = 56312231384.9
Loss = lad, n_estimators = 500, max_depth = 3, learning_rate = 0.1 precision = 52.89 % , error = 54551045303.2
Loss = lad, n_estimators = 1000, max_depth = 3, learning_rate = 0.1 precision = 53.92 % , error = 53354819402.9
Loss = lad, n_estimators = 2000, max_depth = 3, learning_rate = 0.1 precision = 54.97 % , error = 52137025108.6
L

In [20]:
min_error = float('inf')
max_precision = 0
tupla_min_error = ()
tupla_max_precision = ()
for r in res:
    if r[5] < min_error:
        min_error = r[5]
        tupla_min_error = r
    if abs(r[4]) > max_precision:
        max_precision = r[4]
        tupla_max_precision = r
        
print("Mayor precision = Loss = {}, n_estimators = {}, max_depth = {}, learning_rate = {} precision = {:.2f} % , error = {}".\
              format(tupla_max_precision[0],tupla_max_precision[1],tupla_max_precision[2],tupla_max_precision[3], tupla_max_precision[4], tupla_max_precision[5]))
print("Menor error = Loss = {}, n_estimators = {}, max_depth = {}, learning_rate = {} precision = {:.2f} % , error = {}".\
              format(tupla_min_error[0],tupla_min_error[1],tupla_min_error[2],tupla_min_error[3], tupla_min_error[4], tupla_min_error[5]))

Mayor precision = Loss = ls, n_estimators = 100, max_depth = 20, learning_rate = 0.3 precision = 98.39 % , error = 1861890273.38
Menor error = Loss = ls, n_estimators = 100, max_depth = 20, learning_rate = 0.3 precision = 98.39 % , error = 1861890273.38


In [23]:
gradient.score(set_entrenamiento_datos,set_entrenamiento_resultado) * 100

99.408854513852148

# calculo con los verdaderos datos a analizar

In [24]:
analizar = pd.read_csv("../properati_dataset_modificado.csv")

In [25]:
analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: gradient.predict([x])[0],axis = 1)

In [26]:
analizar.price_usd.describe()

count    1.416600e+04
mean     2.453845e+05
std      5.195269e+05
min     -1.403059e+04
25%      1.001735e+05
50%      1.532872e+05
75%      2.590019e+05
max      4.682592e+07
Name: price_usd, dtype: float64

In [27]:
# Las predicciones con precio negativo las convierto en valores positivos
analizar.loc[:,'price_usd'] = analizar.loc[:,"price_usd"].apply(lambda x: abs(x))

In [28]:
analizar.price_usd.describe()

count    1.416600e+04
mean     2.453879e+05
std      5.195253e+05
min      2.138798e+03
25%      1.001735e+05
50%      1.532872e+05
75%      2.590019e+05
max      4.682592e+07
Name: price_usd, dtype: float64

In [29]:
resultado = analizar.loc[:,['id','price_usd']]

In [30]:
resultado.to_csv('resultados/Gradient_boosting_latlon.csv', index = False)

# Usando Grid Search y Cross Validation

In [2]:
propiedades = pd.read_csv('/home/mati/Desktop/set_datos_propiedades.csv')

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2016)]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

In [3]:
# Parametros

loss = ['ls', 'lad', 'huber', 'quantile']
n_estimators = [10,20,50,100]
learning = [0.3, 0.5, 0.7]

parametros = {"loss" : loss, "n_estimators" : n_estimators, "learning_rate" : learning}

iteraciones_cross_validation = 15
gb = GradientBoostingRegressor()

In [4]:
inicio = time.strftime("%X")

grid = GridSearchCV( estimator = gb, param_grid = parametros, n_jobs = 2, cv = iteraciones_cross_validation)

grid.fit(set_entrenamiento_datos, set_entrenamiento_resultado)

score = grid.best_score_ * 100
mejores_parametros = grid.best_params_
fin = time.strftime("%X")

print("Tiempo: {} --- {} \n Precision: {:.2f} \n Parametros = {}".format(inicio,fin,score,mejores_parametros))

Tiempo: 21:32:45 --- 04:47:13 
 Precision: 75.65 
 Parametros = {'learning_rate': 0.5, 'loss': 'ls', 'n_estimators': 100}


In [5]:
analizar = pd.read_csv("/home/mati/Desktop/properati_dataset_modificado.csv")

analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: grid.predict(x)[0],axis = 1)

resultado = analizar.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Gradient_Boosting_GridSearch.csv', index = False)

joblib.dump(grid, 'algoritmos/gradient_boosting.pkl')

['algoritmos/gradient_boosting.pkl']

## Con cross validation

In [2]:
propiedades = pd.read_csv('/home/mati/Desktop/set_datos_propiedades.csv')

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2016)]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

set_pruebas = analizar = pd.read_csv("/home/mati/Desktop/properati_dataset_modificado.csv")
set_pruebas.loc[:,'price_usd'] = 0.0

In [3]:
cant = 10

for i in range(cant):
    datos = propiedades.sample(frac = 0.4)
    set_entrenamiento_datos = datos.loc[:,columnas]
    set_entrenamiento_resultado = datos.loc[:,'price_aprox_usd']
    
    gb = GradientBoostingRegressor(loss = 'ls', n_estimators = 100, learning_rate = 0.5)
    
    gb.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[:,'res'] = set_pruebas.loc[:,columnas].apply(lambda x: gb.predict(x)[0],axis = 1)
    set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] + set_pruebas.loc[:,'res']

In [4]:
set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] / cant

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Gradient_Boosting_Cross_Validation.csv', index = False)